In [ ]:
import os

training_dataset = "cars_priors"
use_lora = False
use_pretrained_weights = True
use_depth = True
use_normal = True
model_path = "/content/gaussian-splatting/experiments_out/"

hf_token = ""
repo_id = "MVP-Group-Project/splatter-image-priors"

In [ ]:

# Clone necessary repos
!git clone --recursive https://github.com/Kacper-M-Michalik/splatter-image.git
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git

In [ ]:
# Install model requirements
%cd /content/splatter-image
!pip install -r requirements.txt

In [ ]:
# Get correct package versions
%cd /content/gaussian-splatting
os.environ['TORCH_CUDA_ARCH_LIST'] = "7.0;7.5;8.0+PTX"
!pip install -e submodules/diff-gaussian-rasterization

In [ ]:
# Import necessary packages and verify versions
from pathlib import Path
from google.colab import drive
import torch
import numpy as np
import matplotlib.pyplot as plt
from huggingface_hub import login, HfApi

print("PyTorch Version:", torch.__version__)
print("CUDA Version:", torch.version.cuda)

In [ ]:
# Finetune existing model to use priors
!python /content/splatter-image/train_network.py +dataset={training_dataset} opt.pretrained_hf={use_pretrained_weights} opt.lora_finetune={use_lora} data.use_pred_depth={use_depth} data.use_pred_normal={use_normal}

In [ ]:
# Save weights to huggingface

files = []
model_path = Path(model_path)

# If model data is directly in folder, use that, otherwise we assume the wandb file structure
possible_files = [file for file in model_path.glob('*{}'.format(".pth"))]
if len(possible_files) > 0:
    files = [file for file in model_path.iterdir() if file.is_file()]
else:
    date_folders = sorted([folder for folder in model_path.iterdir() if folder.is_dir()])
    
    if not date_folders:
        print("No date folders found.")
    else:
        latest_date_folder = date_folders[-1]
        print("Latest Date: {}".format(latest_date_folder.name))

        time_folders = sorted([folder for folder in latest_date_folder.iterdir() if folder.is_dir()])        
        if not time_folders:
            print("No time folders found inside {}".format(latest_date_folder.name))
        else:  
            latest_time_folder = time_folders[-1]
            print(f"Latest Time: {latest_time_folder.name}")
            files = [p for p in latest_time_folder.iterdir() if p.is_file()]

login(token=hf_token)
api = HfApi()

# Upload model
for file_path in files:    
    # Title upload folder correctly
    path_in_repo = "model"
    if use_depth:
        path_in_repo += "-depth"
    if use_normal:
        path_in_repo += "-normal"
    if use_lora:
        path_in_repo += "-finetune"    

    path_in_repo += "/" + os.path.basename(file_path)

    # Upload file
    api.upload_file(
        path_or_fileobj=file_path,
        repo_id=repo_id,
        repo_type="model",
        path_in_repo=path_in_repo,
    )